<a href="https://colab.research.google.com/github/MurilloMMartins/piquete_pitico/blob/main/piquete_pitico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Piquete Pitico
Vítor Fróis

# Install and Configure Colab

In [ ]:
%pip install cohere
%pip install annoy
%pip install altair
%pip install umap-learn

In [ ]:
import json
kj = {"username":"vafrois","key":"12aeb8fccef7590f6f58cb8221eb65cc"}
json.dump(kj, open("kaggle.json", "w"))

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
import cohere
import pandas as pd
import sklearn
import kaggle
import numpy as np
import umap.umap_ as umap
import altair as alt
from annoy import AnnoyIndex

Get Cohere API

In [ ]:
co = cohere.Client('ZnVjqglgjDqpr22J7XMuIG5EjBjjtpJybVH0w5LD')

# Similarity Metrics

### Cosine Similarity

In [ ]:
def cos_sim(a, b):
    return (a @ b.T) / (np.linalg.norm(a)*np.linalg.norm(b))

### Nearest Neighbors Search
Ignore por hora

In [ ]:
class SearchIndex:
  def create_search_index(self, embeds):
    # Create the search index, pass the size of embedding
    search_index = AnnoyIndex(np.array(embeds).shape[1], 'angular')
    # Add all the vectors to the search index
    for i in range(len(embeds)):
      search_index.add_item(i, embeds[i])
    search_index.build(10) # 10 trees
    search_index.save('test.ann')
    self.search_index = search_index

  def get_index(self, item_text, n_ranks=10):
    similar_item_ids = self.search_index.get_nns_by_vector(item_text,n_ranks,
                                                  include_distances=True)
    return similar_item_ids


# Visualize Dataset

## Spotify Dataset

In [ ]:
!kaggle datasets download imuhammad/audio-features-and-lyrics-of-spotify-songs
!unzip /content/audio-features-and-lyrics-of-spotify-songs.zip

audio-features-and-lyrics-of-spotify-songs.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/audio-features-and-lyrics-of-spotify-songs.zip
replace spotify_songs.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
spodf = pd.read_csv("spotify_songs.csv")
spodf.head()

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-10.068,1,0.0236,0.27900,0.01170,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en


In [ ]:
# Lendo os dados
spodf = pd.read_csv('spotify_songs.csv', sep = ',')
# filtrando
spodf.drop_duplicates(subset='track_name', inplace=True)
spodf = spodf[spodf['language'] == 'en']
spodf = spodf[['track_name', 'track_artist', 'lyrics','playlist_genre', 'language', 'track_popularity', 'danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', ]]
spodf = spodf[spodf['track_popularity'] > 50]
spotify_df = spodf[spodf['lyrics'].apply(lambda x: x.isascii())].head(5000)
spotify_df['lyrics'] = spotify_df['lyrics'].str.replace("'", "")
spotify_df.head()

,track_name,track_artist,lyrics,playlist_genre,language,track_popularity,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence
5,Soldier,James TW,"Hold your breath, dont look down, keep trying ...",r&b,en,70,0.496,0.639,0.0550,0.2800,0.00000,0.0975,0.305
6,Satisfy You,Diddy,All I want is somebody whos gonna love me for ...,r&b,en,52,0.764,0.594,0.1850,0.5910,0.00000,0.1450,0.695
11,Limestone,Magic City Hippies,How many friends are you gonna set on fire? Ho...,pop,en,58,0.706,0.457,0.0383,0.2820,0.00849,0.0614,0.723
18,Secrets,The Weeknd,"Everybody here wants you My love, my love And ...",r&b,en,71,0.665,0.771,0.0533,0.0155,0.00001,0.4600,0.764
25,Juke Box Hero,Foreigner,"Standing in the rain, with his head hung low C...",pop,en,67,0.357,0.653,0.0654,0.0828,0.00000,0.0844,0.522


In [ ]:
#spotify_list = ["  ".join([f"{col}: {val}" for col, val in row.items()]) for _, row in spotify_df.iterrows()]
spotify_list = []
for _, row in spotify_df.iterrows():
  aux = "  ".join([f"{col}: {val}" for col, val in row[:4].items()])
  if row['danceability'] > 0.5:
    aux += '  danceable'

  if row['energy'] > 0.5:
    aux += '  energetic'

  if row['speechiness'] > 0.33:
    aux += '  speechable'

  if row['acousticness'] > 0.5:
    aux += '  acoustic'

  if row['instrumentalness'] > 0.25:
    aux += '  instrumental'

  if row['liveness'] > 0.8:
    aux += '  lively'

  aux += '  feeling: '
  if row['valence'] > 0.75:
    aux += 'positive'
  elif row['valence'] < 0.25:
    aux += 'negative'
  else:
    aux += 'neutral'

  spotify_list.append(aux)

spotify_list[:1]

['track_name: Soldier  track_artist: James TW  lyrics: Hold your breath, dont look down, keep trying Darling, its okay to be scared, its frightening At times it just feels like the world is trying hard to knock us down But theres a reason that were still here but no one else is around How weve grown Every single day Im proud I swear, I wont Let anything stop us now Im not a soldier But Ill fight through our darkest of days Get on my shoulders And Ill carry you all of the wayCause theres no life worth living in If youre not with me in it Im not a soldier But you make me wanna be brave Yeah, you do Through the depths of despair, weve stumbled And all that distance between us, weve crumbled Weve ignored all of the rumours and jealous disapprovals And weve come out the other side with something beautiful Oh, how weve grown Every single day Im proud And now I know Theres nothing that can stop us now Im not a soldier But Ill fight through our darkest of days Get on my shoulders And Ill carry

In [ ]:
len(spotify_list)

4316

## Flickr Dataset

In [ ]:
! kaggle datasets download hsankesara/flickr-image-dataset/ -f flickr30k_images/results.csv
! unzip results.csv.zip

 56% 2.00M/3.59M [00:00<00:00, 2.96MB/s]
100% 3.59M/3.59M [00:00<00:00, 4.14MB/s]
Archive:  results.csv.zip
  inflating: results.csv             


In [ ]:
raw_flickr_df = pd.read_csv("results.csv", sep="|")
raw_flickr_df.head()

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .


In [ ]:
# filtering by selected images
selected_images = [
    '1001545525.jpg',
    '1001633352.jpg',
    '100444898.jpg',
    '1011572216.jpg',
    '101362133.jpg',
    '1027149103.jpg',
    '104136873.jpg',
    '104763189.jpg',
    '106027774.jpg',
    '1100214449.jpg',
    '1107507919.jpg'
    ]

# image_desc_df = image_desc_df.loc[image_desc_df["image_name"].isin(selected_images)]
image_desc_df = raw_flickr_df.head(500)
aux = image_desc_df.loc[image_desc_df[' comment_number'] == " 0"]
desc_df = aux[["image_name", " comment"]]
desc_df.columns = ["image_name", "comment"]
desc_df

,image_name,comment
0,1000092795.jpg,Two young guys with shaggy hair look at their...
5,10002456.jpg,Several men in hard hats are operating a gian...
10,1000268201.jpg,A child in a pink dress is climbing up a set ...
15,1000344755.jpg,Someone in a blue shirt and hat is standing o...
20,1000366164.jpg,"Two men , one in a gray shirt , one in a blac..."
...,...,...
475,102851549.jpg,A little boy looks at the camera while a woma...
480,102860573.jpg,A construction site on a street with three me...
485,10287332.jpg,Two men sitting on the roof of a house while ...
490,1028982826.jpg,"Two men , one younger guy with a yellow shirt..."


In [ ]:
list_desc = list(desc_df['comment'])
list_desc[:5]

[' Two young guys with shaggy hair look at their hands while hanging out in the yard .',
 ' Several men in hard hats are operating a giant pulley system .',
 ' A child in a pink dress is climbing up a set of stairs in an entry way .',
 ' Someone in a blue shirt and hat is standing on stair and leaning against a window .',
 ' Two men , one in a gray shirt , one in a black shirt , standing near a stove .']

# Cohere  

### Summarize

In [ ]:
text = """
Come wit' it now
The microphone explodes, shattering the molds
Either drop the hits like De La O or get the fuck off the commode
Wit' the sure shot, sure to make the bodies drop
Drop and don't copy yo, don't call this a co-op
Terror rains drenchin', quenchin' the thirst of the power dons
That five sided fist-a-gon
The rotten sore on the face of mother earth gets bigger
The triggers cold empty ya purse
Rally 'round the family with a pocket full of shells
They rally 'round the family with a pocket full of shells
"""

In [ ]:
response = co.summarize(
    text=text,
    model='command',
    length='short',
    extractiveness='high'
)

summary = response.summary

In [ ]:
summary

'The microphone explodes, shattering the molds. Either drop the hits or get off the commode. The triggers cold empty ya purse.'

### Embed

In [ ]:
spotify_embeddings = co.embed(texts=spotify_list,
                  model="embed-english-v2.0").embeddings

In [ ]:
len(spotify_embeddings)

4316

In [ ]:
spotify_df['embeddings'] = spotify_embeddings
spotify_df

,track_name,track_artist,lyrics,playlist_genre,language,track_popularity,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,embeddings
5,Soldier,James TW,"Hold your breath, dont look down, keep trying ...",r&b,en,70,0.496,0.639,0.0550,0.2800,0.000000,0.0975,0.305,"[1.1474609, 0.2130127, 0.019760132, 1.6064453,..."
6,Satisfy You,Diddy,All I want is somebody whos gonna love me for ...,r&b,en,52,0.764,0.594,0.1850,0.5910,0.000000,0.1450,0.695,"[0.3503418, 0.9199219, -0.4519043, 1.296875, 2..."
11,Limestone,Magic City Hippies,How many friends are you gonna set on fire? Ho...,pop,en,58,0.706,0.457,0.0383,0.2820,0.008490,0.0614,0.723,"[-0.27978516, -0.14672852, -0.09863281, 0.1464..."
18,Secrets,The Weeknd,"Everybody here wants you My love, my love And ...",r&b,en,71,0.665,0.771,0.0533,0.0155,0.000010,0.4600,0.764,"[-0.90234375, 1.1425781, 0.009765625, 0.300781..."
25,Juke Box Hero,Foreigner,"Standing in the rain, with his head hung low C...",pop,en,67,0.357,0.653,0.0654,0.0828,0.000000,0.0844,0.522,"[1.8154297, 1.2851562, 0.09008789, 1.5361328, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18435,Pusherman,Curtis Mayfield,Im your Mama Im your Daddy Im that nigga In th...,r&b,en,55,0.851,0.669,0.0379,0.2390,0.085200,0.1080,0.911,"[-0.064575195, 0.040496826, -0.49169922, 0.983..."
18439,The Next Time I Fall (with Amy Grant),Peter Cetera,"(Verse 1) Love, like a road that never ends Ho...",rock,en,57,0.656,0.347,0.0280,0.4880,0.002210,0.0645,0.368,"[0.8198242, 0.2529297, 0.0065345764, 1.3925781..."
18440,Cold Water,Major Lazer,"Everybody gets high sometimes, you know What e...",pop,en,69,0.608,0.798,0.0432,0.0736,0.000000,0.1560,0.501,"[-0.61621094, 1.5351562, -0.8466797, 2.1152344..."
18447,Sick Feeling,boy pablo,"I had to catch the bus Alarm was shaking, oh A...",rock,en,64,0.425,0.504,0.0314,0.0220,0.000004,0.3630,0.403,"[0.9746094, -0.059295654, -0.4428711, 1.603515..."


In [ ]:
spotify_df.to_csv("embeddings.csv")

In [ ]:
spotify_si = SearchIndex()
spotify_si.create_search_index(spotify_embeddings)

In [ ]:
desc_to_compare = list_desc[0]
desc_to_compare = 'there is a man in a suit and sunglasses holding a gun  feeling: neutral'

In [ ]:
desc_embeddings = co.embed(texts=[desc_to_compare],
                  model="embed-english-v2.0").embeddings

In [ ]:
similar_item_ids = spotify_si.get_index(desc_embeddings[0], 10) #n_ranks = 10 by default

In [ ]:
similar_item_ids

([4169, 1389, 3076, 150, 1620, 3900, 210, 688, 2957, 4304],
 [1.0895825624465942,
  1.0920244455337524,
  1.0936329364776611,
  1.103674054145813,
  1.1176170110702515,
  1.1302164793014526,
  1.133726954460144,
  1.1372144222259521,
  1.1377992630004883,
  1.1425644159317017])

In [ ]:
results = pd.DataFrame(data={'texts': spotify_df.iloc[similar_item_ids[0]]['track_name'],
                             'artist': spotify_df.iloc[similar_item_ids[0]]['track_artist'],
                             'distance': similar_item_ids[1]})
print(f"Question:'{desc_to_compare}'\nNearest neighbors:")
results

Question:'there is a man in a suit and sunglasses holding a gun  feeling: neutral'
Nearest neighbors:


,texts,artist,distance
17717,I'm a Wanted Man,Royal Deluxe,1.089583
5241,Dirty Harry,Gorillaz,1.092024
12704,9 Crimes,Damien Rice,1.093633
529,Shoot to Thrill,AC/DC,1.103674
6244,Pull The Trigger,Russ,1.117617
16326,Banana Clip,Miguel,1.130216
752,Sharp Dressed Man,ZZ Top,1.133727
2506,Absolute Zero,Stone Sour,1.137214
12146,Answer to No One (feat. JJ Lawhorn),Colt Ford,1.137799
18400,Clint Eastwood,Gorillaz,1.142564


In [ ]:
# UMAP reduces the dimensions from 1024 to 2 dimensions that we can plot
reducer = umap.UMAP(n_neighbors=20)
umap_embeds = reducer.fit_transform(spotify_embeddings)
# Prepare the data to plot and interactive visualization
# using Altair
df_explore = pd.DataFrame(data={'text': list(spotify_df['track_name'])})
df_explore['x'] = umap_embeds[:,0]
df_explore['y'] = umap_embeds[:,1]

# Plot
chart = alt.Chart(df_explore).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    tooltip=['text']
).properties(
    width=700,
    height=400
)
chart.interactive()

alt.Chart(...)

In [ ]:
umap_embeds

array([[14.705049 , 10.942977 ],
       [12.525688 , 10.587846 ],
       [15.743431 ,  9.642721 ],
       [13.377213 , 12.390102 ],
       [12.421109 , 12.195867 ],
       [10.440864 , 11.987578 ],
       [12.454265 , 10.675991 ],
       [12.997838 , 11.768911 ],
       [14.893143 ,  9.241576 ],
       [13.519932 , 12.102614 ],
       [14.730202 , 10.65235  ],
       [11.422017 , 10.92684  ],
       [13.104654 , 10.991671 ],
       [11.43557  ,  9.714071 ],
       [14.041692 , 10.586151 ],
       [12.378629 , 12.6118765],
       [13.463734 ,  9.677325 ],
       [13.85192  , 11.942844 ],
       [14.047768 , 10.533659 ],
       [10.80893  , 11.588397 ],
       [11.713448 , 12.307165 ],
       [12.737368 , 11.458694 ],
       [15.394175 ,  9.236326 ],
       [14.10842  ,  9.778006 ],
       [11.870093 , 11.390021 ],
       [12.774734 , 10.28437  ],
       [12.032433 , 11.110862 ],
       [11.3498745, 11.789375 ],
       [11.593693 , 11.718281 ],
       [13.152157 , 10.009303 ],
       [11

# Tests with Segmentation

In [ ]:
img1 = 'there is a man sitting at a table with a laptop computer'
img2 = 'smiling man with two fingers up in front of a laptop'
img3 = 'there are two men sitting at a table with laptops'
img4 = 'there is a person sitting at a table with a laptop computer'
emotion = 'neutral'
img_total = img1+img2+img3+img4+emotion

In [ ]:
desc_embeddings = co.embed(texts=[img_total],
                  model="embed-english-v2.0").embeddings

'there is a man sitting at a table with a laptop computersmiling man with two fingers up in front of a laptopthere are two men sitting at a table with laptopsthere is a person sitting at a table with a laptop computer'

In [ ]:
response = co.chat(
	"Please recommend a music to post in a Instragram Stories photo with the following description: there is a man sitting at a table with a laptop computer neutral",
	model="command",
	temperature=0.9
)

answer = response.text


In [ ]:
answer

'Here are a few music recommendations that may fit well with a photo of a man sitting at a table with a laptop computer:\n\n- "Digital Age" by Hello Meteor (3:03)\n- "Lite Years" by Josh Rose (2:36)\n- "Daylight" by Nick Miller (3:08)\n- "Home" by Cautious Clay (3:14)\n- "Everything Else" by Delta (3:55)\n\nThese songs have a neutral, upbeat feel to them and may help create a positive and engaging atmosphere for your Instagram story.'